In [4]:
import os
import torch
from pathlib import Path
!pip install xarray netCDF4
import xarray
import netCDF4
HOME = Path.home()
print("using home dir:", HOME)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print("\nCuda is available: "+str(torch.cuda.is_available()))

!nvcc --version

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
using home dir: /home/ec2-user
Thu Jun  8 13:53:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   25C    P8    10W / 300W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+--------------------

In [87]:
# download data from s3
import boto3
import netCDF4
import os
import boto3
import sagemaker
import xarray as xr
import pandas as pd
from pathlib import Path
HOME=Path.home()
%alias mkdatadir mkdir -p %l
sess = sagemaker.Session()
bucket = sess.default_bucket()                    # Set a default S3 bucket
print(f"Using s3 bucket: {bucket}")
DATA_DIR=f"{HOME}/SageMaker/data_downloads"
MODEL_DATA=f"{HOME}/SageMaker/model_data"
print(f"Using download data path: {DATA_DIR}")
print(f"Using model data path: {DATA_DIR}")
%mkdatadir $DATA_DIR

Using s3 bucket: sagemaker-us-west-2-014690315417
Using download data path: /home/ec2-user/SageMaker/data_downloads
Using model data path: /home/ec2-user/SageMaker/data_downloads


In [ ]:
# download the data from s3


def sync_s3_to_local(bucket_name, prefix, local_directory):
    s3 = boto3.client('s3')

    # List all objects in the bucket with the specified prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    objects = response['Contents']

    # Sync each object to the local directory
    for obj in objects:
        key = obj['Key']
        local_path = os.path.join(local_directory, key)

        # Create the local directory if it doesn't exist
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        # Check if the local file already exists
        if os.path.exists(local_path):
            print(f"Skipping {key} (already exists)")
            continue

        # Download the object from S3 to the local directory
        s3.download_file(bucket_name, key, local_path)
        print(f"Downloaded {key}")

# Usage example
prefix = "data/era5_labeled_regridded"
sync_s3_to_local(bucket,prefix,  DATA_DIR)


In [149]:
import glob
source_directory = '/home/ec2-user/SageMaker/data_downloads/data/era5_labeled_regridded'
train_directory = f'{MODEL_DATA}/train'
val_directory = f'{MODEL_DATA}/val'
test_directory = f'{MODEL_DATA}/test'

%mkdatadir $train_directory
%mkdatadir $val_directory
%mkdatadir $test_directory


val_pattern = '2018-12-*.nc'
test_pattern = '2018-11-*.nc'
print(MODEL_DATA, train_directory)
# Get the list of files in the source directory matching the pattern
# val and test should not overlap
val_files = glob.glob(os.path.join(source_directory, file_pattern))
test_files = glob.glob(os.path.join(source_directory, test_pattern))
train_files = [f for f in all_files if f not in val_files and f not in test_files]
# Create symbolic links for each file in the target directory

n_train, n_val, n_test = 1, 1, 1

train_files=train_files[0:n_train]
val_files=val_files[0:n_val]
test_files=test_files[0:n_test]


# Iterate over the files and remove symbolic links
print("unlinking existing files")
for directory in [train_directory, val_directory, test_directory]:
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.islink(file_path):
            os.unlink(file_path)

print("linking downloads to model data")
for file in train_files:
    source_path = file
    target_path = os.path.join(train_directory, os.path.basename(file))
    os.symlink(source_path, target_path)

for file in val_files:
    source_path = file
    target_path = os.path.join(val_directory, os.path.basename(file))
    os.symlink(source_path, target_path)

for file in test_files:
    source_path = file
    target_path = os.path.join(test_directory, os.path.basename(file))
    os.symlink(source_path, target_path)


print("done")
# # for file in val_files:
#     source_path = os.path.join(source_directory, file)
#     target_path = os.path.join(val_directory, file)
#     os.symlink(source_path, target_path)
! echo Training files && ls $train_directory
print()
! echo Validation files && ls $val_directory
print()
! echo Test files && ls $test_directory

/home/ec2-user/SageMaker/model_data /home/ec2-user/SageMaker/model_data/train
unlinking existing files
linking downloads to model data
done
Training files
2018-01-01.nc

Validation files
2018-12-08.nc

Test files
2018-11-18.nc


In [160]:
%cd $HOME/SageMaker/ClimateNet
%pwd
JACCARD_PATH = "'./models/Baseline'"
%ls $JACCARD_PATH
%cat $JACCARD_PATH/config.json

/home/ec2-user/SageMaker/ClimateNet
config.json  history.csv  weights.pth
{
    "architecture": "cgnet",
    "seed": 42,
    "lr": 0.001,
    "train_batch_size": 4,
    "pred_batch_size": 8,
    "epochs": 15,
    "fields": {
        "TMQ": {
            "mean": 19.21859,
            "std": 15.81723
        },
        "U850": {
            "mean": 1.55302,
            "std": 8.29764
        },
        "V850": {
            "mean": 0.25413,
            "std": 6.23163
        },
        "PSL": {
            "mean": 100814.414,
            "std": 1461.2227
        }
    },
    "labels": [
        "Background",
        "Tropical Cyclone",
        "Atmospheric River"
    ],
    "loss": "jaccard",
    "scheduler": false,
    "description": "Baseline: CGNet with jaccard loss trained on original ClimateNet with no learning rate scheduling."
}


In [161]:
!python3 train_eval.py \
-d $MODEL_DATA \
-m $HOME/SageMaker/ClimateNet/$JACCARD_PATH/


================ Training epoch #1 (cuda:0) ================
Loss: 0.88028 (jaccard) | LR: 0.001:   0%|                | 0/1 [00:01<?, ?it/s]/home/ec2-user/SageMaker/ClimateNet/climatenet/models.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({'minibatch_loss': train_loss.item()}, ignore_index=True)
Loss: 0.88028 (jaccard) | LR: 0.001: 100%|████████| 1/1 [00:02<00:00,  2.17s/it]
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:62: RuntimeWarning: invalid value encountered in divide
  recall = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:64: RuntimeWarning: invalid value encountered in divide
  sensitivity = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/models.py:137: FutureWarning: The frame.append method is deprecated and will be